In [1]:
import pandas as pd
import numpy as np
# 设置工作目录
import os
address = r'E:\ggq的文件\比赛\24统计建模'
os.chdir(address)

## data文件夹项下数据整理  
data  
a：证券交易所提供的原始公司列表  
│  a上交所主板600_.xlsx  
│  a上交所科创板688_.xlsx  
│  a深交所主板000_.xlsx  
│  a深交所创业板300_.xlsx  
│  a上市公司地区分类列表_上海证券交易所.xlsx  
│  a上市公司行业分类列表_上海证券交易所.xlsx  
b：经过整理后的公司列表，包含所属省份和行业  
│  b上交所主板.xlsx  
│  b上交所科创板.xlsx  
│  b深交所主板.xlsx  
│  b深交所创业板.xlsx  
c：新浪财经年度报告链接（不全）  
│  c上交所主板_年报链接.xlsx  
│  c上交所科创板_年报链接.xlsx   
│  c深交所主板_年报链接.xlsx   
│  c深交所创业板_年报链接.xlsx  
d：新浪财经年度报告链接（所有可获得）  
│  d上交所主板_年报链接全.xlsx  
│  d上交所科创板_年报链接全.xlsx  
│  d深交所主板_年报链接全.xlsx  
│  d深交所创业板_年报链接全.xlsx  
e：最终筛选后的公司列表（drop ST金融科研）  
│  final_公司年报列表_dropST金融科研_去不平衡_28483条_1994家企业.xlsx  
│  final_公司年报列表_dropST金融科研_去不平衡_29051条_2033家企业.xlsx  
f：GPT打分结果  
│  gptscore_2022.xlsx  
z：其他  
|  zCSMAR企业数字化转型库.xlsx

In [2]:
def read_xlsx(address, symbol, year):
    data = pd.read_excel(address, dtype={symbol: str})
    data.columns = data.columns.str.strip()   # 删除空格
    data[symbol] = data[symbol].astype(str).str.extract(r'(\d+)')   # 删除标点和英文字符
    data[symbol] = data[symbol].astype(str).str.zfill(6)    # 规范股票代码格式
    data = data.rename(columns={symbol: '股票代码'})
    # 如果year非空
    if year:
        data = data.rename(columns={year: '年份'})
    print("形状：", data.shape)
    print("列名：", data.columns)
    print("包含公司数量：", len(data['股票代码'].unique()))
    return data

def print_df(df):
    print("形状：", df.shape)
    print("列名：", df.columns)
    print("包含公司数量：", len(df['股票代码'].unique()))
    
def print_col(df):
    for column in df.columns:
        print(column, df[column].dtype)
        
def missing_values(df):
    grouped = df.groupby('股票代码')
    missing_values_count = grouped.apply(lambda x: x.isnull().sum())
    missing_values = missing_values_count.groupby(level=0).sum()
    print(missing_values)
    return missing_values

def check_list(df_1, col_1, df_2, col_2):
    """
    检查DataFrame df_1 中指定列 col_1 的每个唯一值是否存在于 DataFrame df_2 的指定列 col_2 中。
    """
    unique_values = df_1[col_1].unique()
    reference_values = df_2[col_2].unique()
    missing_values = list(set(unique_values) - set(reference_values))
    return missing_values

In [3]:
# 完整上市公司列表
all_company_df = pd.DataFrame()
all_company_file_list = ['b上交所主板.xlsx', 'b上交所科创板.xlsx', 'b深交所主板.xlsx', 'b深交所创业板.xlsx']  
for i in range (len(all_company_file_list)):
    df_address = 'data/'+all_company_file_list[i]
    df = read_xlsx(df_address, 'A股代码', None)
    all_company_df = pd.concat([all_company_df, df], axis=0)
print_df(all_company_df)

形状： (1696, 7)
列名： Index(['股票代码', 'A股简称', '英文名称', 'A股上市日期', '省份', '所属行业', '板块'], dtype='object')
包含公司数量： 1696
形状： (571, 7)
列名： Index(['股票代码', 'A股简称', '英文名称', 'A股上市日期', '省份', '所属行业', '板块'], dtype='object')
包含公司数量： 571
形状： (1503, 9)
列名： Index(['股票代码', 'A股简称', '英文名称', 'A股上市日期', '地区', '省份', '城市', '所属行业', '板块'], dtype='object')
包含公司数量： 1503
形状： (1343, 9)
列名： Index(['股票代码', 'A股简称', '英文名称', 'A股上市日期', '地区', '省份', '城市', '所属行业', '板块'], dtype='object')
包含公司数量： 1343
形状： (5113, 9)
列名： Index(['股票代码', 'A股简称', '英文名称', 'A股上市日期', '省份', '所属行业', '板块', '地区', '城市'], dtype='object')
包含公司数量： 5113


## regress文件夹项下数据整理
a：绿色生产率和AI词频  
│  a人工智能词频AIDIC.xlsx  
│  a绿色生产率.xlsx  
b：控制变量1  
│  b控制变量-人员构成.xlsx  
│  b控制变量-企业创新.xlsx  
│  b控制变量-企业绿色专利.xlsx  
│  b控制变量-企业财务信息.xlsx   
│  b控制变量-常用.xlsx  
│  b控制变量-环保投资营收比.xlsx  
│  b控制变量-环保投资资产比.xlsx  
│  b控制变量-融资约束SA指数.xlsx  
c：控制变量2  
│  c控制变量-专利与创新效率.xlsx  
│  c控制变量-企业创新投入.xlsx  
│  c控制变量-企业创新质量.xlsx  
│  c控制变量-平均创新质量.xlsx  
│  c控制变量-持续绿色创新水平.xlsx  
│  c控制变量-整体创新质量.xlsx  
│  c控制变量-绿色创新效率.xlsx  
│  c控制变量-绿色科技研发成果转化效率.xlsx  
│  c控制变量-绿色管理创新.xlsx  

In [4]:
# 常用控制变量
df_address = 'regress/b控制变量-常用.xlsx '
control_regular = read_xlsx(df_address, '股票代码', '年份')
columns = [
    "股票代码", "年份", "ShortName", "当年是否ST或PT", "样本区间内是否ST或PT",	"样本区间内是否退市", "Province", "City",
    "Size", "Lev", "Cashflow", "FIXED", "Growth", "Board", "Dual", "Top1", "TobinQ",
    "FirmAge", "Employee", "REC", "INV", "Intangible", "AssetGrowth", "NetProfitGrowth",
    "ITR", "CAP", "Invest1", "Indep", "Balance1", "Seperate", "BM", "PB", "Ofee",
    "Mfee", "ATO", "SA", "WW", "FC", "KZ", "East", "West", "Mid", "HighTech_1",
    "HighTech_2", "HighTech_3", "HighTech_4", "HighTech_5", "Pollute_1", "Pollute_2",
    "Pollute_3", "SOE", "Loss"
]
control_regular = control_regular[columns]

形状： (57522, 123)
列名： Index(['股票代码', '年份', 'ShortName', '当年是否ST或PT', '样本区间内是否ST或PT', '样本区间内是否退市',
       '金融业', '制造业', '沪深上市', '北京上市',
       ...
       'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5', 'Pollute_1',
       'Pollute_2', 'Pollute_3', '劳动密集型', '技术密集型', '资产密集型'],
      dtype='object', length=123)
包含公司数量： 5338


In [5]:
# 绿色控制变量
control_green = control_regular[['股票代码', '年份']].copy()
# 整体创新质量LnCit1, 平均创新质量LnCit2, 绿色创新效率GreenInnov, 企业绿色科技研发效率grd和绿色科技成果转化效率gcon, 创新投入持续性IIP和创新产出持续性OIP
control_green_file_list = ['c控制变量-整体创新质量.xlsx', 'c控制变量-平均创新质量.xlsx', 'c控制变量-绿色创新效率.xlsx', 'c控制变量-绿色科技研发成果转化效率.xlsx', 'c控制变量-持续绿色创新水平.xlsx  ']
for i in range (len(control_green_file_list)):
    df_address = 'regress/'+control_green_file_list[i]
    df = read_xlsx(df_address, '股票代码', '年份')
    control_green = pd.merge(control_green, df, on=['股票代码', '年份'], how='left')
print_df(control_green)

形状： (36634, 3)
列名： Index(['股票代码', '年份', 'LnCit1'], dtype='object')
包含公司数量： 3961
形状： (36634, 3)
列名： Index(['股票代码', '年份', 'LnCit2'], dtype='object')
包含公司数量： 3961
形状： (31039, 3)
列名： Index(['年份', '股票代码', 'GreenInnov'], dtype='object')
包含公司数量： 4464
形状： (46408, 4)
列名： Index(['股票代码', '年份', 'grd', 'gcon'], dtype='object')
包含公司数量： 4581
形状： (59989, 4)
列名： Index(['年份', '股票代码', 'IIP', 'OIP'], dtype='object')
包含公司数量： 5299
形状： (57522, 9)
列名： Index(['股票代码', '年份', 'LnCit1', 'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP',
       'OIP'],
      dtype='object')
包含公司数量： 5338


In [6]:
gtfp = read_xlsx('regress/a绿色生产率.xlsx', '股票代码', '年份')
aidic = read_xlsx('regress/a人工智能词频AIDIC.xlsx', '股票代码', '年份')
aigpt = read_xlsx('data/gptscore_2022.xlsx', '股票代码', '年份')

形状： (46419, 5)
列名： Index(['股票代码', '年份', '企业绿色全要素生产率', '绿色技术效率变化指数', '绿色技术进步变化指数'], dtype='object')
包含公司数量： 4581
形状： (55354, 5)
列名： Index(['股票代码', '年份', '全文-文本总长度', '仅中英文-文本总长度', 'AIDIC'], dtype='object')
包含公司数量： 5155
形状： (1994, 7)
列名： Index(['AIGPT', '股票代码', '年份', 'A股简称', '所属行业', '板块', '省份'], dtype='object')
包含公司数量： 1994


In [7]:
gtfp = gtfp[['股票代码', '年份', '企业绿色全要素生产率']]
gtfp = gtfp.rename(columns={'企业绿色全要素生产率': 'GTFP'})
aidic = aidic[['股票代码', '年份', 'AIDIC']]
aidic['AIDIC_累加后log'] = aidic.groupby('股票代码')['AIDIC'].cumsum()
aidic['AIDIC_累加后log'] = np.log(aidic['AIDIC_累加后log'] + 1)
aigpt = aigpt[['股票代码', '年份', 'AIGPT']]

In [8]:
control_df = pd.merge(control_regular, control_green, on=['股票代码', '年份'], how='left')
explain_df = pd.merge(control_df, gtfp, on=['股票代码', '年份'], how='left')
explain_df = pd.merge(explain_df, aidic, on=['股票代码', '年份'], how='left')
explain_df = pd.merge(explain_df, aigpt, on=['股票代码', '年份'], how='left')
all_df = pd.merge(explain_df, all_company_df, on=['股票代码'], how='left')
print_df(all_df)

形状： (57522, 71)
列名： Index(['股票代码', '年份', 'ShortName', '当年是否ST或PT', '样本区间内是否ST或PT', '样本区间内是否退市',
       'Province', 'City', 'Size', 'Lev', 'Cashflow', 'FIXED', 'Growth',
       'Board', 'Dual', 'Top1', 'TobinQ', 'FirmAge', 'Employee', 'REC', 'INV',
       'Intangible', 'AssetGrowth', 'NetProfitGrowth', 'ITR', 'CAP', 'Invest1',
       'Indep', 'Balance1', 'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO',
       'SA', 'WW', 'FC', 'KZ', 'East', 'West', 'Mid', 'HighTech_1',
       'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5', 'Pollute_1',
       'Pollute_2', 'Pollute_3', 'SOE', 'Loss', 'LnCit1', 'LnCit2',
       'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'GTFP', 'AIDIC',
       'AIDIC_累加后log', 'AIGPT', 'A股简称', '英文名称', 'A股上市日期', '省份', '所属行业', '板块',
       '地区', '城市'],
      dtype='object')
包含公司数量： 5338


In [9]:
ID_list = aigpt['股票代码'].unique().tolist()

In [10]:
# 打印filtered_df各列数据类型
print_col(all_df)

股票代码 object
年份 int64
ShortName object
当年是否ST或PT int64
样本区间内是否ST或PT int64
样本区间内是否退市 int64
Province object
City object
Size float64
Lev float64
Cashflow float64
FIXED float64
Growth float64
Board float64
Dual int64
Top1 float64
TobinQ float64
FirmAge float64
Employee float64
REC float64
INV float64
Intangible float64
AssetGrowth float64
NetProfitGrowth float64
ITR float64
CAP float64
Invest1 float64
Indep float64
Balance1 float64
Seperate float64
BM float64
PB float64
Ofee float64
Mfee float64
ATO float64
SA float64
WW float64
FC float64
KZ float64
East float64
West float64
Mid float64
HighTech_1 int64
HighTech_2 int64
HighTech_3 int64
HighTech_4 int64
HighTech_5 int64
Pollute_1 int64
Pollute_2 int64
Pollute_3 int64
SOE float64
Loss int64
LnCit1 float64
LnCit2 float64
GreenInnov float64
grd float64
gcon float64
IIP float64
OIP float64
GTFP float64
AIDIC float64
AIDIC_累加后log float64
AIGPT float64
A股简称 object
英文名称 object
A股上市日期 object
省份 object
所属行业 object
板块 object
地区 object
城市 object


In [11]:
filtered_df = all_df[all_df['股票代码'].isin(ID_list)]
filtered_df = filtered_df[(filtered_df['年份'] >= 2007) & (filtered_df['年份'] <= 2022)]
print_df(filtered_df)

形状： (29489, 71)
列名： Index(['股票代码', '年份', 'ShortName', '当年是否ST或PT', '样本区间内是否ST或PT', '样本区间内是否退市',
       'Province', 'City', 'Size', 'Lev', 'Cashflow', 'FIXED', 'Growth',
       'Board', 'Dual', 'Top1', 'TobinQ', 'FirmAge', 'Employee', 'REC', 'INV',
       'Intangible', 'AssetGrowth', 'NetProfitGrowth', 'ITR', 'CAP', 'Invest1',
       'Indep', 'Balance1', 'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO',
       'SA', 'WW', 'FC', 'KZ', 'East', 'West', 'Mid', 'HighTech_1',
       'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5', 'Pollute_1',
       'Pollute_2', 'Pollute_3', 'SOE', 'Loss', 'LnCit1', 'LnCit2',
       'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'GTFP', 'AIDIC',
       'AIDIC_累加后log', 'AIGPT', 'A股简称', '英文名称', 'A股上市日期', '省份', '所属行业', '板块',
       '地区', '城市'],
      dtype='object')
包含公司数量： 1994


In [12]:
filtered_df = filtered_df.drop(columns=['当年是否ST或PT', '样本区间内是否ST或PT', '样本区间内是否退市', '省份', '地区', '城市', 'A股简称', '英文名称'])
filtered_df = filtered_df.rename(columns={'Province': '省份', 'City': '城市', 'ShortName': 'A股简称'})

In [13]:
# AI_{i,t}=AIDIC_{i,t}\ast\frac{AIGPT_{i,2022}}{AIDIC_{i,2022}}
# 对于股票代码分组，计算每一组2022年的AIGPT除以AIDIC，然后将这个值乘到AIDIC上，得到新列AI
ratios = filtered_df[filtered_df['年份'] == 2022].groupby('股票代码').apply(lambda x: x['AIGPT'].iloc[0] / x['AIDIC_累加后log'].iloc[0])
filtered_df['AI'] = filtered_df.apply(lambda x: x['AIDIC_累加后log'] * ratios[x['股票代码']], axis=1)
# filtered_df['AI'] = filtered_df['AI'].fillna(0)

C:\Users\25105\AppData\Local\Temp\ipykernel_21552\3996110076.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  ratios = filtered_df[filtered_df['年份'] == 2022].groupby('股票代码').apply(lambda x: x['AIGPT'].iloc[0] / x['AIDIC_累加后log'].iloc[0])
C:\Users\25105\AppData\Local\Temp\ipykernel_21552\3996110076.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ratios = filtered_df[filtered_df['年份'] == 2022].groupby('股票代码').apply(lambda x: x['AIGPT'].iloc[0] / x['AIDIC_累加后log'].iloc[0])
C:\Users\25105\AppData\Local\Temp\ipykernel_21552\3996110076.py:4: RuntimeWarning: invalid value encountered in scalar multiply
  filtered_df['AI'] = filtered_df.apply(lambda x: x['AIDIC_累加后log'] * ratios[x[

In [14]:
# 重排列的顺序
columns = ['股票代码', '年份', 'A股简称', 'A股上市日期', '省份', '城市', '所属行业', '板块', 'GTFP', 'AIDIC', 'AIDIC_累加后log', 'AIGPT', 'AI',
       'East', 'West', 'Mid', 
       'LnCit1', 'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP',
       'Size', 'Lev', 'Cashflow', 'FIXED', 'Growth',
       'Board', 'Dual', 'Top1', 'TobinQ', 'FirmAge', 'Employee', 'REC', 'INV',
       'Intangible', 'AssetGrowth', 'NetProfitGrowth', 'ITR', 'CAP', 'Invest1',
       'Indep', 'Balance1', 'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO',
       'SA', 'WW', 'FC', 'KZ',  'HighTech_1', 'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5', 
       'Pollute_1', 'Pollute_2', 'Pollute_3', 'SOE', 'Loss']
filtered_df = filtered_df[columns]

In [15]:
# 将nan和inf填充为0
filtered_df = filtered_df.replace([np.inf, -np.inf], np.nan)
filtered_df = filtered_df.fillna(0)

In [17]:
print_df(filtered_df)

形状： (29489, 64)
列名： Index(['股票代码', '年份', 'A股简称', 'A股上市日期', '省份', '城市', '所属行业', '板块', 'GTFP',
       'AIDIC', 'AIDIC_累加后log', 'AIGPT', 'AI', 'East', 'West', 'Mid', 'LnCit1',
       'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'Size', 'Lev',
       'Cashflow', 'FIXED', 'Growth', 'Board', 'Dual', 'Top1', 'TobinQ',
       'FirmAge', 'Employee', 'REC', 'INV', 'Intangible', 'AssetGrowth',
       'NetProfitGrowth', 'ITR', 'CAP', 'Invest1', 'Indep', 'Balance1',
       'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO', 'SA', 'WW', 'FC', 'KZ',
       'HighTech_1', 'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5',
       'Pollute_1', 'Pollute_2', 'Pollute_3', 'SOE', 'Loss'],
      dtype='object')
包含公司数量： 1994


In [18]:
# 1 99缩尾处理
from scipy.stats.mstats import winsorize
df_list=['GTFP', 'AIDIC', 'AIDIC_累加后log', 'AIGPT', 'AI', 
       'East', 'West', 'Mid', 'LnCit1', 'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'Size', 'Lev',
       'Cashflow', 'FIXED', 'Growth', 'Board', 'Dual', 'Top1', 'TobinQ',
       'FirmAge', 'Employee', 'REC', 'INV', 'Intangible', 'AssetGrowth',
       'NetProfitGrowth', 'ITR', 'CAP', 'Invest1', 'Indep', 'Balance1',
       'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO', 'SA', 'WW', 'FC', 'KZ',
       'HighTech_1', 'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5',
       'Pollute_1', 'Pollute_2', 'Pollute_3', 'SOE', 'Loss']#需要进行缩尾的列名
winsorize_df = filtered_df.copy()
for i in df_list:
    winsorize_df[i]=winsorize(winsorize_df[i],limits=[0.01, 0.01])

In [19]:
missing_df = missing_values(winsorize_df)

        股票代码  年份  A股简称  A股上市日期  省份  城市  所属行业  板块  GTFP  AIDIC  ...  \
股票代码                                                           ...   
000002     0   0     0       0   0   0     0   0     0      0  ...   
000006     0   0     0       0   0   0     0   0     0      0  ...   
000008     0   0     0       0   0   0     0   0     0      0  ...   
000010     0   0     0       0   0   0     0   0     0      0  ...   
000011     0   0     0       0   0   0     0   0     0      0  ...   
...      ...  ..   ...     ...  ..  ..   ...  ..   ...    ...  ...   
603333     0   0     0       0   0   0     0   0     0      0  ...   
603366     0   0     0       0   0   0     0   0     0      0  ...   
603399     0   0     0       0   0   0     0   0     0      0  ...   
603766     0   0     0       0   0   0     0   0     0      0  ...   
603993     0   0     0       0   0   0     0   0     0      0  ...   

        HighTech_1  HighTech_2  HighTech_3  HighTech_4  HighTech_5  Pollute_1  \
股票代码    

C:\Users\25105\AppData\Local\Temp\ipykernel_21552\2764837871.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_values_count = grouped.apply(lambda x: x.isnull().sum())


In [20]:
missing_df.to_excel('regress/missing.xlsx', index=True)
winsorize_df.to_excel('regress/d实证数据_筛选1994家企业_29489条数据.xlsx', index=False)

## 哑变量

In [42]:
regress_df = read_xlsx('regress/d实证数据_筛选1994家企业_29489条数据.xlsx', '股票代码', '年份')
politics = read_xlsx('regress/e人工智能政策受益.xlsx', '股票代码', None)
shichang = pd.read_excel('regress/e地市市场化分割指数.xlsx')
print(shichang.columns)
gdp = pd.read_excel('regress/e城市GDP.xlsx')
print(gdp.columns)

形状： (29489, 64)
列名： Index(['股票代码', '年份', 'A股简称', 'A股上市日期', '省份', '城市', '所属行业', '板块', 'GTFP',
       'AIDIC', 'AIDIC_累加后log', 'AIGPT', 'AI', 'East', 'West', 'Mid', 'LnCit1',
       'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'Size', 'Lev',
       'Cashflow', 'FIXED', 'Growth', 'Board', 'Dual', 'Top1', 'TobinQ',
       'FirmAge', 'Employee', 'REC', 'INV', 'Intangible', 'AssetGrowth',
       'NetProfitGrowth', 'ITR', 'CAP', 'Invest1', 'Indep', 'Balance1',
       'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO', 'SA', 'WW', 'FC', 'KZ',
       'HighTech_1', 'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5',
       'Pollute_1', 'Pollute_2', 'Pollute_3', 'SOE', 'Loss'],
      dtype='object')
包含公司数量： 1994
形状： (18658, 22)
列名： Index(['序号', '代码_链接', '股票代码', '名称_链接', '名称', '相关_链接', '相关', '相关_链接1', '相关2',
       '最新价', '今日涨跌幅', '今日主力净流入', '今日超大单净流入', '今日大单净流入', '今日中单净流入', '今日小单净流入',
       '字段', '字段3', '字段4', '字段5', '字段6', '字段22'],
      dtype='object')
包含公司数量： 4499
Index(['省份', '地级市', '

In [43]:
missing_values1 = check_list(regress_df, '城市', shichang, '地级市')
missing_values2 = check_list(regress_df, '省份', gdp, '省份')
print(missing_values1)
print(missing_values2)
missing_chengshi = regress_df[regress_df['城市'].isin(missing_values1)]
missing_chengshi = missing_chengshi[['城市', '省份']].drop_duplicates().reset_index(drop=True)
print(missing_chengshi.head())

['龙口市', '库尔勒市', '普宁市', '廊坊市', '新沂市', '安宁市', '林州市', '樟树市', '涿州市', '当阳市', '平顶山市', '集安市', '江阴市', '双河市', '衡水市', '荆州市', '宜兴市', '峨眉山市', '吉首市', '招远市', '新郑市', '高密市', '昌邑市', '项城市', '汾阳市', '胶州市', '如皋市', '义马市', '诸城市', '仁怀市', '余姚市', '临海市', '海东市', '江山市', '兴平市', '海门市', '临安市', '永安市', '兰溪市', '靖江市', '浏阳市', '灵武市', '宜春市', '锡林郭勒盟', '凌海市', '宜宾市', '武穴市', '汉川市', '南安市', '什邡市', '禹城市', '应城市', '凌源市', '寿光市', '启东市', '三河市', '泰州市', '即墨市', '武安市', '海宁市', '温岭市', '介休市', '西昌市', '常熟市', '沙河市', '瑞安市', '滁州市', '荣成市', '平湖市', '巢湖市', '建德市', '霍林郭勒市', '海安市', '东阳市', '青铜峡市', '慈溪市', '义乌市', '镇江市', '澄迈县', '宁国市', '和田市', '鹤山市', '玉环市', '福泉市', '沅江市', '奉化市', '曲阜市', '诸暨市', '邹城市', '江油市', '张家港市', '永城市', '丽江市', '永康市', '兴宁市', '昆山市', '福清市', '阜阳市', '贵溪市', '丹阳市', '丰城市', '晋江市', '广州市', '长葛市', '太仓市', '乐清市', '广安市', '巩义市', '仪征市', '林芝市', '孟州市', '敦化市', '偃师市', '柳州市', '阿克苏市', '莱州市', '桐乡市', '格尔木市', '昌吉市', '洛阳市']
[]
     城市   省份
0   海安市  江苏省
1   宜春市  江西省
2  格尔木市  青海省
3   广州市  广东省
4   寿光市  山东省


In [44]:
shichang = shichang[shichang['年份'] == 2022]
shichang = shichang[['省份', '地级市', '年份', '市场分割指数', '市场一体化指数']]
shichang = shichang.rename(columns={'地级市': '城市'})
# 拼接
shichang = pd.concat([shichang, missing_chengshi], axis=0)
# 按照省份分组，将'市场分割指数', '市场一体化指数'的缺失值补充为组内均值
shichang['市场分割指数'] = shichang.groupby('省份')['市场分割指数'].transform(lambda x: x.fillna(x.mean()))
shichang['市场一体化指数'] = shichang.groupby('省份')['市场一体化指数'].transform(lambda x: x.fillna(x.mean()))
# shichang数据框生成哑变量1：市场分割指数均值切分，以上为1，以下为0
shichang['市场分割指数均值切分'] = shichang['市场分割指数'].apply(lambda x: 1 if x >= shichang['市场分割指数'].mean() else 0)
# shichang数据框生成哑变量2：市场一体化指数均值切分，以上为1，以下为0
shichang['市场一体化指数均值切分'] = shichang['市场一体化指数'].apply(lambda x: 1 if x >= shichang['市场一体化指数'].mean() else 0)
print(shichang.columns)

Index(['省份', '城市', '年份', '市场分割指数', '市场一体化指数', '市场分割指数均值切分', '市场一体化指数均值切分'], dtype='object')


In [45]:
gdp = gdp[gdp['年份'] == 2022]
gdp = gdp[['省份', '年份', '人均GDP(元)']]
# gdp数据框生成哑变量3：人均gdp均值切分，以上为1，以下为0
gdp['人均gdp均值切分'] = gdp['人均GDP(元)'].apply(lambda x: 1 if x >= gdp['人均GDP(元)'].mean() else 0)
print(gdp.columns)

Index(['省份', '年份', '人均GDP(元)', '人均gdp均值切分'], dtype='object')


In [46]:
politics = politics[['股票代码']]
# DID
politics['人工智能政策受益'] = 1
# 删除重复行
politics = politics.drop_duplicates().reset_index(drop=True)

In [47]:
# 将哑变量merge到regress_df
regress_df = pd.merge(regress_df, shichang[['城市', '市场分割指数均值切分', '市场一体化指数均值切分']], on='城市', how='left')
regress_df = pd.merge(regress_df, gdp[['省份', '人均gdp均值切分']], on='省份', how='left')
regress_df = pd.merge(regress_df, politics, on='股票代码', how='left')
print(regress_df['市场分割指数均值切分'].isnull().sum())
print(regress_df['市场一体化指数均值切分'].isnull().sum())
print(regress_df['人均gdp均值切分'].isnull().sum())
regress_df['人工智能政策受益'] = regress_df['人工智能政策受益'].fillna(0)

0
0
0


In [48]:
regress_df.to_excel('regress/e实证数据_筛选1994家企业_29489条数据_哑变量.xlsx', index=False)

## 劳动力结构

In [11]:
address = r'E:\ggq的文件\比赛\24统计建模\regress\e实证数据_筛选1994家企业_29489条数据_哑变量.xlsx'
regress_df = read_xlsx(address, '股票代码', '年份')
# address = r'E:\ggq的文件\比赛\24统计建模\regress\e人员构成RESSET_EMPINFO_1.xlsx'
# emp1 = read_xlsx(address, 'A股股票代码_A_StkCd', None)
# address = r'E:\ggq的文件\比赛\24统计建模\regress\e人员构成RESSET_EMPINFO_2.xlsx'
# emp2 = read_xlsx(address, 'A股股票代码_A_StkCd', None)
# emp = pd.concat([emp1, emp2], axis=0)
address = r'E:\ggq的文件\比赛\24统计建模\regress\e人员构成CSRR_CompanyStaff.xlsx'
emp = read_xlsx(address, '股票代码', None)
check_list(regress_df, '股票代码', emp,'股票代码')

形状： (29489, 68)
列名： Index(['股票代码', '年份', 'A股简称', 'A股上市日期', '省份', '城市', '所属行业', '板块', 'GTFP',
       'AIDIC', 'AIDIC_累加后log', 'AIGPT', 'AI', 'East', 'West', 'Mid', 'LnCit1',
       'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'Size', 'Lev',
       'Cashflow', 'FIXED', 'Growth', 'Board', 'Dual', 'Top1', 'TobinQ',
       'FirmAge', 'Employee', 'REC', 'INV', 'Intangible', 'AssetGrowth',
       'NetProfitGrowth', 'ITR', 'CAP', 'Invest1', 'Indep', 'Balance1',
       'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO', 'SA', 'WW', 'FC', 'KZ',
       'HighTech_1', 'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5',
       'Pollute_1', 'Pollute_2', 'Pollute_3', 'SOE', 'Loss', '市场分割指数均值切分',
       '市场一体化指数均值切分', '人均gdp均值切分', '人工智能政策受益'],
      dtype='object')
包含公司数量： 1994
形状： (863111, 13)
列名： Index(['股票代码', '统计截止日期', '数据来源', '口径编码', '人员结构编码', '序号', '股票简称', '公告日期',
       '口径名称', '人员结构', '人员明细', '数量', '单位'],
      dtype='object')
包含公司数量： 5582


[]

In [12]:
ID_list = regress_df['股票代码'].unique().tolist()
emp = emp[emp['股票代码'].isin(ID_list)]
print(emp.shape)
print(emp['人员结构'].unique())
emp = emp[emp['人员结构'].isin(['总人数', '专业结构'])]
print(emp.shape)
emp['统计截止日期'] = pd.to_datetime(emp['统计截止日期'])
emp['年份'] = emp['统计截止日期'].apply(lambda x: x.year)
emp = emp[['股票代码', '年份','人员结构编码', '序号', '人员结构', '人员明细', '数量', '单位']]

(498261, 13)
['总人数' '学历' '性别' '民族' '专业结构' '年龄' '所在地区' '其他' '职称' '职业资格' '入职年限']
(315192, 13)


In [15]:
# 查看'人员结构编码'与'人员结构'的对应关系
structure_code_to_structure = emp.groupby('人员结构编码')['人员结构'].unique().apply(list).to_dict()
# 查看'人员结构编码'与'人员明细'的对应关系
structure_code_to_detail = emp.groupby('人员结构编码')['人员明细'].unique().apply(list).to_dict()
print(structure_code_to_structure)
# 把structure_code_to_detail导出为json，存在的汉字使用utf-8编码
import json
with open('E:/ggq的文件/比赛/24统计建模/regress/e人员结构_编码to明细.json', 'w', encoding='utf-8') as f:
    json.dump(structure_code_to_detail, f, ensure_ascii=False)

{'P5701': ['总人数'], 'P5709': ['专业结构']}


In [17]:
emp_1 = emp[(emp['人员结构编码'] == 'P5701') & (emp['序号'] == 1)]
print(emp_1[emp_1['人员结构编码'] == 'P5701']['人员明细'].unique())

['在册员工' '员工总数' '在职员工' '总人数' '报告期末在职员工的数量合计' '共有在册员工' '母公司员工总数'
 '母公司在职员工的数量' '\ufeff母公司在职员工的数量' '报告期末母公司在职员工的数量' '母公司在职员工' '现有员工' '员工总人数'
 '总部在职员工' '共有员工' '在职员工的数量合计' '主要子公司在职员工的数量' '在职员工人数' '母公司在职员工数量'
 '子公司在职员工数量' '公司需承担费用的离退休职工人数' '从业人员' '从业人员总数' '在职员工总数' '退休员工' '员工人数'
 '职工合计' '员工合计' '从业人员合计' '在册员工(未含控股子公司)' '在职员工的人数' '全球员工总数' '在岗员工总数'
 '母公司员工人数' '需承担费用的离退休职工人数' '公司共有员工' '公司总员工总数' '母公司在职人数' '在职正式员工'
 '公司员工人数合计' '公司在职职工总数' '期末从业人员人数' '员工数量' '集团员工总数' '公司及所属企业员工' '在册职工'
 '在岗职工' '本公司(包括控股子公司)共有在册员工' '公司在册员工人数' '在岗员工总人数' '公司需承担费用的退休人员' '离退休职工'
 '当期领取薪酬员工总人数' '在职员工总人数' '公司需承担费用的离退休人数' '公司在职员工' '凯傲公司人数' '全球劳动合同员工'
 '共有劳动合同员工' '拥有员工' '雇员总数' '在职员工数量' '在册员工总数' '在职员工的人' '职工总数'
 '母公司及主要子公司需承担费用的离退休职工人数' '在编员工总数' '员工' '在职职工总数' '在岗员工人数' '公司员工总数'
 '全体员工人数' '在册职工人数' '在岗员工' '公司及主要子公司在岗员工' '公司共有在职员工' '公司及主要子公司共有在职员工'
 '公司在册员工' '公司承担费用的离退休职工人数' '报告期末公司本部在职员工数量' '公司本部在职职工数量' '从业人员数'
 '母公司及主要子公司在职员工的人数' '在职员工数量总数' '公司需承担费用的离退休人员为' '在岗职工总数' '合同制员工' '公司现有员工'
 '在职职工人数' '在册职工总数' '公司在职人数' '公司现有员工总数' '公司员工数量'

In [ ]:
# 删除包含'退'、'女'的人员明细


## 工具变量

In [56]:
address = r'E:\ggq的文件\比赛\24统计建模\regress\e全球AI专利.xlsx'
patent = pd.read_excel(address)
print(patent.columns)
address = r'E:\ggq的文件\比赛\24统计建模\regress\e地市1984年邮局数和固定电话数.xlsx'
post = pd.read_excel(address)
print(post.columns)
check_list(regress_df, '省份', post,'所属省份')

Index(['Entity', 'Code', 'Year', 'Patent applications - Field: All'], dtype='object')
Index(['行政区划代码', '地区', '所属省份', '长江经济带', '经度', '纬度', '1984年末总人口数（万人）',
       '1984年末邮电局数（处）', '1984年末电话机数（部）', '1984年末邮电业务总量（万元）', '1984年每百万人邮局数',
       '1984年每百人固定电话数'],
      dtype='object')


['西藏自治区']

In [57]:
post = post[['所属省份', '1984年每百人固定电话数']]
# 按照省份分组并平均，得到截面数据框
post = post.groupby('所属省份').mean().reset_index()
print(post.head())

     所属省份  1984年每百人固定电话数
0     上海市       2.467471
1     云南省       0.421092
2  内蒙古自治区       1.087957
3     北京市       4.079234
4     吉林省       1.087668


In [58]:
patent = patent[['Year', 'Patent applications - Field: All']]
patent = patent.rename(columns={'Year': '年份', 'Patent applications - Field: All': '专利申请数'})
patent['专利申请数_t-1'] = patent['专利申请数'].shift(-1)
patent['专利申请数_t-1_log'] = np.log(patent['专利申请数_t-1'])
patent = patent.drop(columns=['专利申请数'])
patent = patent.dropna()
print(patent.head())

     年份  专利申请数_t-1  专利申请数_t-1_log
0  2022   115838.0      11.659948
1  2021    93828.0      11.449219
2  2020    74418.0      11.217453
3  2019    53303.0      10.883748
4  2018    32029.0      10.374397


In [59]:
from itertools import product
provinces = post['所属省份'].tolist()
# 添加['西藏自治区']到provinces
provinces.append('西藏自治区')
years = patent['年份'].tolist()
combined_list = list(product(provinces, years))
panel_data = pd.DataFrame(combined_list, columns=['所属省份', '年份'])
print(panel_data)

      所属省份    年份
0      上海市  2022
1      上海市  2021
2      上海市  2020
3      上海市  2019
4      上海市  2018
..     ...   ...
491  西藏自治区  2011
492  西藏自治区  2010
493  西藏自治区  2009
494  西藏自治区  2008
495  西藏自治区  2007

[496 rows x 2 columns]


In [60]:
# 合并截面数据
panel_data = panel_data.merge(post, on='所属省份', how='left')
panel_data = panel_data.merge(patent[['年份', '专利申请数_t-1_log']], on='年份', how='left')
# 计算工具变量
panel_data['工具变量'] = panel_data['1984年每百人固定电话数'] * panel_data['专利申请数_t-1_log']
panel_data = panel_data.rename(columns={'所属省份': '省份'})
panel_data = panel_data.fillna(0)
print(panel_data.head())

    省份    年份  1984年每百人固定电话数  专利申请数_t-1_log       工具变量
0  上海市  2022       2.467471      11.659948  28.770586
1  上海市  2021       2.467471      11.449219  28.250618
2  上海市  2020       2.467471      11.217453  27.678743
3  上海市  2019       2.467471      10.883748  26.855335
4  上海市  2018       2.467471      10.374397  25.598526


In [62]:
address = r'E:\ggq的文件\比赛\24统计建模\regress\e实证数据_筛选1994家企业_29489条数据_哑变量.xlsx'
regress_df = read_xlsx(address, '股票代码', '年份')
# 工具变量合并
regress_df = regress_df.merge(panel_data[['省份', '年份', '工具变量']], on=['省份', '年份'], how='left')
regress_df.to_excel('regress/e实证数据_筛选1994家企业_29489条数据_哑变量_工具变量.xlsx', index=False)

形状： (29489, 68)
列名： Index(['股票代码', '年份', 'A股简称', 'A股上市日期', '省份', '城市', '所属行业', '板块', 'GTFP',
       'AIDIC', 'AIDIC_累加后log', 'AIGPT', 'AI', 'East', 'West', 'Mid', 'LnCit1',
       'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'Size', 'Lev',
       'Cashflow', 'FIXED', 'Growth', 'Board', 'Dual', 'Top1', 'TobinQ',
       'FirmAge', 'Employee', 'REC', 'INV', 'Intangible', 'AssetGrowth',
       'NetProfitGrowth', 'ITR', 'CAP', 'Invest1', 'Indep', 'Balance1',
       'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO', 'SA', 'WW', 'FC', 'KZ',
       'HighTech_1', 'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5',
       'Pollute_1', 'Pollute_2', 'Pollute_3', 'SOE', 'Loss', '市场分割指数均值切分',
       '市场一体化指数均值切分', '人均gdp均值切分', '人工智能政策受益'],
      dtype='object')
包含公司数量： 1994


## 劳动力结构_垃圾版

In [64]:
address = r'E:\ggq的文件\比赛\24统计建模\regress\b控制变量-人员构成.xlsx'
labor = read_xlsx(address, '股票代码', '年份')
labor = labor[['股票代码', '年份', '生产占比', '销售占比', '财务占比', '技术占比', '其他部门占比']]
labor['常规低技能劳动力占比'] = labor['生产占比'] + labor['销售占比'] + labor['财务占比']
labor['非常规高技能劳动力占比'] = labor['技术占比']
address = r'E:\ggq的文件\比赛\24统计建模\regress\e实证数据_筛选1994家企业_29489条数据_哑变量.xlsx'
regress_df = read_xlsx(address, '股票代码', '年份')
# labor变量合并
regress_df = regress_df.merge(labor[['股票代码', '年份', '常规低技能劳动力占比', '非常规高技能劳动力占比']], on=['股票代码', '年份'], how='left')
regress_df.to_excel('regress/e实证数据_筛选1994家企业_29489条数据_哑变量_工具变量_劳动力.xlsx', index=False)

形状： (62176, 26)
列名： Index(['年份', '股票代码', '证券简称', '员工总数', '人均创收万元', '人均创利万元', '研究生以上学历',
       '研究生及以上学历占比', '本科', '本科学历占比', '专科以上学历', '专科以上学历占比', '高中及以下学历',
       '高中及以下学历占比', '其他学历', '其他学历占比', '生产', '生产占比', '财务', '财务占比', '销售', '销售占比',
       '技术', '技术占比', '其他部门', '其他部门占比'],
      dtype='object')
包含公司数量： 5224
形状： (29489, 68)
列名： Index(['股票代码', '年份', 'A股简称', 'A股上市日期', '省份', '城市', '所属行业', '板块', 'GTFP',
       'AIDIC', 'AIDIC_累加后log', 'AIGPT', 'AI', 'East', 'West', 'Mid', 'LnCit1',
       'LnCit2', 'GreenInnov', 'grd', 'gcon', 'IIP', 'OIP', 'Size', 'Lev',
       'Cashflow', 'FIXED', 'Growth', 'Board', 'Dual', 'Top1', 'TobinQ',
       'FirmAge', 'Employee', 'REC', 'INV', 'Intangible', 'AssetGrowth',
       'NetProfitGrowth', 'ITR', 'CAP', 'Invest1', 'Indep', 'Balance1',
       'Seperate', 'BM', 'PB', 'Ofee', 'Mfee', 'ATO', 'SA', 'WW', 'FC', 'KZ',
       'HighTech_1', 'HighTech_2', 'HighTech_3', 'HighTech_4', 'HighTech_5',
       'Pollute_1', 'Pollute_2', 'Pollute_3', 'SOE', 'Loss', '市场分割